In [ ]:
!pip install contextily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from sklearn.cluster import DBSCAN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/5g_hack/5g_6g hackathon/unique_towers.csv")
print(df.shape)

(239, 2)


In [ ]:

df=pd.read_csv("/content/drive/MyDrive/5g_hack/5g_6g hackathon/unique_towers.csv")

fig, ax = plt.subplots(figsize=(10, 8))
for i in range(len(df)):
    ax.scatter(
        df.iloc[i, 0],  # Longitude
        df.iloc[i, 1],  # Latitude
        # marker='^',     # Triangle marker
        s=100,          # Size of the marker, adjust as needed
        alpha=0.8,      # Transparency of the marker
        color='orange'    # Color of the marker
    )
ax.set_xlim([12.28, 12.63])  # Longitude range
ax.set_ylim([41.78, 42])      # Latitude range

    # Add OSM basemap
ctx.add_basemap(ax, crs="EPSG:4326", source=ctx.providers.OpenStreetMap.Mapnik)

    # Add titles and labels
    # ax.set_title(f'Cluster Points on OSM Map - Hour {hour}')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.savefig(os.path.join( f'cells.png'))
plt.close()  # Close the plot to avoid display

In [ ]:
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on the Earth's surface using the Haversine formula.

    Parameters:
        lat1, lon1: Latitude and longitude of the first point in degrees
        lat2, lon2: Latitude and longitude of the second point in degrees

    Returns:
        Distance between the two points in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Earth's radius in kilometers (mean radius)
    R = 6371.0

    # Calculate the distance
    distance = R * c
    return distance


In [ ]:
d=5
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx  # For adding basemaps
import os

# Threshold distance
d = 3

# Loop through each hour's CSV data (modify the path as needed)
for i in range(7, 21):
    df_ = pd.read_csv(f"/content/drive/MyDrive/5g_hack/5g_6g hackathon/hour clusters csv values/hourly_median_{i}.csv")
    fig, ax = plt.subplots(figsize=(10, 8))
    flag1=0;
    flag2=0
    flag3=0
    coordinates_set = set()

    # Add the coordinate pair (latitude, longitude) to the set

    for k in range(len(df)):
      flag=0
      for j in range(len(df_)):
        if (df.iloc[j, 0], df.iloc[j, 1]) not in coordinates_set:

          coordinates_set.add((df.iloc[j, 0], df.iloc[j, 1]))
          # print("x: ",df_.iloc[j, 2],"y: ",df_.iloc[j, 1])
          ax.scatter(
              df_.iloc[j, 2],
              df_.iloc[j, 1],
              s=df_.iloc[j, 4] * 10000,  # Scale size, adjust as needed
              alpha=0.3,  # Transparency
              color='blue'
              # label='Median Point' if flag3==0 else ""  # Label only once for the legend
          )
        flag3=1
        d1 = haversine_distance(df_.iloc[j, 1], df_.iloc[j, 2], df.iloc[k, 1], df.iloc[k, 0])
            # Plot towers based on distance threshold
        if d1 < d:
          flag=1
          break

      if flag==0:
         ax.scatter(
                    df.iloc[k, 0],
                    df.iloc[k, 1],
                    marker='^',
                    s=10,  # Scale size, adjust as needed
                    alpha=0.5,  # Transparency
                    color='red',
                    label='Cell outside threshold' if (flag2==0) else ""  # Label only once
                )
         flag2=1
      else:
          ax.scatter(
                    df.iloc[k, 0],
                    df.iloc[k, 1],
                    marker='^',
                    s=10,  # Scale size, adjust as needed
                    alpha=0.5,  # Transparency
                    color='green',
                    label='Cell within threshold' if (flag1==0) else ""  # Label only once
                )
          flag1=1


    # Plot each cluster point


    # Set longitude and latitude limits
    ax.set_xlim([12.28, 12.63])  # Longitude range
    ax.set_ylim([41.78, 42])     # Latitude range

    # Add OSM basemap
    ctx.add_basemap(ax, crs="EPSG:4326", source=ctx.providers.OpenStreetMap.Mapnik)

    # Add titles and labels
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Add legend
    ax.legend(loc='upper right')  # You can adjust the location as needed

    # Save the figure
    plt.savefig(os.path.join(f'/content/drive/MyDrive/5g_hack/5g_6g hackathon/hour clusters csv values/hourly_median_cells{i}.png'))
    plt.close()
